# Artificial Neural Network for Thickness & PEEQ Prediction


In [1]:
# 1. Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import StandardScaler
import joblib   # for saving scalers


KeyboardInterrupt: 

In [ ]:
# 2. Load Data
# ----------------------------------------------------------
train_df = pd.read_csv("test_dataset.csv")
test_df = pd.read_csv("train_dataset.csv")

# Drop non-numeric/identifier columns
drop_cols = ["Type", "Instance", "Label"]
train_df = train_df.drop(columns=drop_cols, errors="ignore")
test_df = test_df.drop(columns=drop_cols, errors="ignore")

# Separate inputs and outputs
X_train = train_df.drop(columns=["Thickness", "PEEQ"])
y_train = train_df[["Thickness", "PEEQ"]]

X_test = test_df.drop(columns=["Thickness", "PEEQ"])
y_test = test_df[["Thickness", "PEEQ"]]

# Fill missing values with 0
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)
y_train = y_train.fillna(0)
y_test = y_test.fillna(0)

In [ ]:
# ----------------------------------------------------------
# 3. Scale Data
# ----------------------------------------------------------
scaler_X = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

scaler_y = StandardScaler()
y_train_scaled = scaler_y.fit_transform(y_train)
y_test_scaled = scaler_y.transform(y_test)


In [ ]:
# ----------------------------------------------------------
# 4. Build ANN Model
# ----------------------------------------------------------
model = Sequential([
    Dense(64, input_dim=X_train_scaled.shape[1], activation='relu'),
    Dense(32, activation='relu'),
    Dense(2, activation='linear')  # Two outputs: Thickness, PEEQ
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])


In [ ]:
# ----------------------------------------------------------
# 5. Train Model
# ----------------------------------------------------------
history = model.fit(
    X_train_scaled, y_train_scaled,
    epochs=100, batch_size=16,
    validation_split=0.2, verbose=1
)


In [ ]:
# ----------------------------------------------------------
# 6. Save Model + Scalers
# ----------------------------------------------------------
model.save("ann_model.keras")
joblib.dump(scaler_X, "scaler_X.pkl")
joblib.dump(scaler_y, "scaler_y.pkl")

print("Model and scalers saved!")

In [ ]:
# ----------------------------------------------------------
# 7. Reload Model + Scalers (for inference)
# ----------------------------------------------------------
model = load_model("ann_model.keras")
scaler_X = joblib.load("scaler_X.pkl")
scaler_y = joblib.load("scaler_y.pkl")


In [ ]:
from keras.models import load_model
model = load_model("ann_model.keras")


In [ ]:
model.compile(optimizer="adam", loss="mse", metrics=["mae", "mse"])


In [ ]:
from tensorflow.keras.metrics import MeanSquaredError, MeanAbsoluteError

model.compile(optimizer="adam",
              loss=MeanSquaredError(),
              metrics=[MeanAbsoluteError(), MeanSquaredError()])


In [ ]:
# ----------------------------------------------------------
# 8. Test on Test Data
# ----------------------------------------------------------
loss, mae = model.evaluate(X_test_scaled, y_test_scaled, verbose=0)
print(f"Test MSE: {loss:.4f}, Test MAE: {mae:.4f}")

y_pred_scaled = model.predict(X_test_scaled)
y_pred = scaler_y.inverse_transform(y_pred_scaled)
y_true = y_test.values


In [ ]:
# ----------------------------------------------------------
# 9. Visualize Results
# ----------------------------------------------------------
for i, col in enumerate(["Thickness", "PEEQ"]):
    plt.figure(figsize=(6, 5))
    plt.scatter(y_true[:, i], y_pred[:, i], alpha=0.7)
    plt.plot([y_true[:, i].min(), y_true[:, i].max()],
             [y_true[:, i].min(), y_true[:, i].max()], 'r--')
    plt.xlabel(f"True {col}")
    plt.ylabel(f"Predicted {col}")
    plt.title(f"True vs Predicted for {col}")
    plt.show()


In [ ]:
# Print sample results
results = pd.DataFrame({
    "True_Thickness": y_true[:, 0],
    "Pred_Thickness": y_pred[:, 0],
    "True_PEEQ": y_true[:, 1],
    "Pred_PEEQ": y_pred[:, 1],
})
print(results.head())
